In [1]:
import pandas as pd

In [2]:
df = pd.read_feather('./master_dataset.ftr')

In [3]:
df

,dhs_id,chr,start,end,DHS_width,summit,numsamples,total_signal,component,proportion,...,fKidney_ENCLB005SRL,fKidney_ENCLB704GMQ,fKidney_ENCLB759USM,fLung_ENCLB594BSZ,fKidney_ENCLB049MNH,fUmbilical_cord_ENCLB771UER,fBone_femur_ENCLB236BWV,fLiver_ENCLB638FEH,fPlacenta_ENCLB423VBC,fPlacenta_ENCLB711ZZZ
0,chr1_16140_16200_16170,chr1,16140,16200,60,16170,1,0.129388,1,0.855153,...,0,0,0,0,0,0,0,0,0,0
1,chr1_51868_52040_51970,chr1,51868,52040,172,51970,1,0.080034,7,0.973545,...,0,0,0,0,0,0,0,0,0,0
2,chr1_57280_57354_57350,chr1,57280,57354,74,57350,4,1.093002,8,1.000000,...,0,0,0,0,0,0,0,0,0,0
3,chr1_66370_66482_66430,chr1,66370,66482,112,66430,8,1.469725,3,0.332213,...,0,0,0,0,0,0,0,0,0,0
4,chr1_79100_79231_79150,chr1,79100,79231,131,79150,2,0.226098,7,0.501840,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3591893,chrY_56882540_56882719_56882610,chrY,56882540,56882719,179,56882610,1,0.038079,5,0.803229,...,0,0,0,0,0,0,0,0,0,0
3591894,chrY_56882864_56882980_56882930,chrY,56882864,56882980,116,56882930,1,0.115489,5,0.742349,...,0,0,0,0,0,0,0,0,0,0
3591895,chrY_56883733_56883960_56883830,chrY,56883733,56883960,227,56883830,5,2.456885,7,0.559734,...,0,0,0,0,0,0,0,0,0,0
3591896,chrY_56884440_56884580_56884510,chrY,56884440,56884580,140,56884510,1,0.053759,5,0.803229,...,0,0,0,0,0,0,0,0,0,0


In [4]:
class FilteringData:
    def __init__(self, df: pd.DataFrame, cell_list: list):
        self.df = df
        self.cell_list = cell_list
        self._test_data_structure()

    def _test_data_structure(self):
        # Ensures all columns after the 11th are named cell names
        assert all('_ENCL' in x for x in self.df.columns[11:]), '_ENCL not in all columns after 11th'

    def filter_exclusive_replicates(self, sort: bool = False, balance: bool = True):
        """Given a specific set of samples (one per cell type),
        capture the exclusive peaks of each samples (the ones matching just one sample for the whole set)
        and then filter the dataset to keep only these peaks.

        Returns:
            pd.DataFrame: The original dataframe plus a column for each cell type with the exclusive peaks
        """
        print('Filtering exclusive peaks between replicates')
        # Selecting the columns corresponding to the cell types
        subset_cols = self.df.columns[:11].tolist() + self.cell_list
        # Creating a new dataframe with only the columns corresponding to the cell types
        df_subset = self.df[subset_cols]
        # Creating a new column for each cell type with the exclusive peaks or 'NO_TAG' if not exclusive
        df_subset['TAG'] = df_subset[self.cell_list].apply(lambda x: 'NO_TAG' if x.sum() != 1 else x.idxmax(), axis=1)

        # Creating a new dataframe with only the rows with exclusive peaks
        new_df_list = []
        for k, v in df_subset.groupby('TAG'):
            if k != 'NO_TAG':
                cell, replicate = '_'.join(k.split('_')[:-1]), k.split('_')[-1]
                v['additional_replicates_with_peak'] = (
                    df[df.filter(like=cell).columns].apply(lambda x: x.sum(), axis=1) - 1
                )
                temp_df = self.df.filter(like=cell)
                print(f'Cell type: {cell}, Replicate: {replicate}, Number of exclusive peaks: {v.shape[0]}')
            else:
                v['additional_replicates_with_peak'] = 0
            new_df_list.append(v)
        new_df = pd.concat(new_df_list).sort_index()
        new_df['other_samples_with_peak_not_considering_reps'] = (
            new_df['numsamples'] - new_df['additional_replicates_with_peak'] - 1
        )

        # Sorting the dataframe by the number of samples with the peak
        if sort:
            new_df = pd.concat(
                [
                    x_v.sort_values(
                        by=['additional_replicates_with_peak', 'other_samples_with_peak_not_considering_reps'],
                        ascending=[False, True],
                    )
                    for x_k, x_v in new_df.groupby('TAG')
                ],
                ignore_index=True,
            )

        # Balancing the dataset
        if balance:
            lowest_peak_count = new_df.groupby('TAG').count()['sequence'].min()
            new_df = pd.concat(
                [v_bal.head(lowest_peak_count) for k_bal, v_bal in new_df.groupby('TAG') if k_bal != 'NO_TAG']
            )

        return new_df

In [7]:
test_df = FilteringData(
    df, cell_list=['K562_ENCLB843GMH', 'hESCT0_ENCLB449ZZZ', 'HepG2_ENCLB029COU', 'GM12878_ENCLB441ZZZ']
).filter_exclusive_replicates(sort=True, balance=True)

Filtering exclusive peaks between replicates


/tmp/ipykernel_37652/303766290.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['TAG'] = df_subset[self.cell_list].apply(lambda x: 'NO_TAG' if x.sum() != 1 else x.idxmax(), axis=1)


Cell type: GM12878, Replicate: ENCLB441ZZZ, Number of exclusive peaks: 11968
Cell type: HepG2, Replicate: ENCLB029COU, Number of exclusive peaks: 73621
Cell type: K562, Replicate: ENCLB843GMH, Number of exclusive peaks: 71106
Cell type: hESCT0, Replicate: ENCLB449ZZZ, Number of exclusive peaks: 202853


In [11]:
test_df.to_feather("filtered_dataset.ftr")

In [6]:
import pickle as pkl
biosample_dict = pkl.load(open('biosample_type_map.pkl', 'rb'))

In [7]:
cell_list=['K562_ENCLB843GMH', 'hESCT0_ENCLB449ZZZ', 'HepG2_ENCLB029COU', 'GM12878_ENCLB441ZZZ']

for c in cell_list:
    print(f'{c} - {biosample_dict[c]}')

K562_ENCLB843GMH - Cancer
hESCT0_ENCLB449ZZZ - Lines
HepG2_ENCLB029COU - Cancer
GM12878_ENCLB441ZZZ - Lines
